1. Попробуйте улучшить работу нейронной сети (разобранную на уроке), обучавшейся на датасет Fashion-MNIST. Напишите в комментариях к уроку, какого результата вы добились от нейросети и что помогло улучшить её точность.
2. Попробуйте найти полезные команды TensorFlow, неразобранные на уроке
3. Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews.
Напишите в комментариях к уроку, какого результата вы добились от нейросети и что
помогло улучшить её точность

In [1]:
!pip install tensorboard
!pip install -q -U keras-tuner
!pip install tensorflow-datasets tensorflow
!pip install tensorflow-hub

План по улучшению:
 - Составить несколько моделей с разными параметрами;
 - Поискать оптимальные варианты;
 - Применить несколько разных оптимайзеров;
 - Применить callback;
 - Сделать сборку предсказаний из нескольких моделей (Идея обучить одну из моделей на обратном векторе);

In [2]:
# from __future__ import absolute_import,division, print_function, unicode_literals

#TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_datasets as tfds
tf.random.set_seed(5)
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical, to_ordinal
from tensorflow.keras.models import load_model

#Helper libraries
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, accuracy_score

print(tf.__version__)

C:\Users\rikks\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend
2.14.0


In [3]:
%tensorboard --logdir logs/fit/

UsageError: Line magic function `%tensorboard` not found.


## Импортируем imdb reviews датасет

In [4]:

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch


<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

Датасет содержит следующие классы:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>neg</td>
  </tr>
  <tr>
    <td>1</td>
    <td>pos</td>
  </tr>
</table>

Каждому классу обозначенному цифрой мы можем присвоить текстовое значение -

In [7]:
class_names = ['neg', 'pos']

Для процесса обучения нейронной сети нам важно перевести данные из диапазона от 0 до 255 в диапазон от 0 до 1:

### Определение слоев

Давайте создадим 3 слоя нейронной сети с помощью функционала Keras.layers

сначала создадим слой Keras, который использует модель TensorFlow Hub для встраивания предложений, и попробуем его на нескольких входных примерах. Обратите внимание, что независимо от длины входного текста, выходная форма вложений: (num_examples, embedding_dimension) .

In [8]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [22]:
model = keras.Sequential([
    hub_layer,
    keras.layers.Dropout(0.1),
    keras.layers.Dense(256, activation='relu', name='hiden_one'),
    keras.layers.Dense(128, activation='relu', name='hiden_two'),
    keras.layers.Dense(64, activation='tanh', name='hiden_three'),
    keras.layers.Dense(64, activation='relu', name='hiden_four'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, name='output')
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dropout_11 (Dropout)        (None, 50)                0         
                                                                 
 hiden_one (Dense)           (None, 256)               13056     
                                                                 
 hiden_two (Dense)           (None, 128)               32896     
                                                                 
 hiden_three (Dense)         (None, 64)                8256      
                                                                 
 hiden_four (Dense)          (None, 64)                4160      
                                                                 
 batch_normalization_6 (Bat  (None, 64)               

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [29]:
history = model.fit(train_data.shuffle(10000).batch(256),
                    epochs=15,
                    validation_data=validation_data.batch(256),
                    verbose=1)


Epoch 1/15


59/59 [==============================] - 32s 517ms/step - loss: 0.0045 - accuracy: 0.9979 - val_loss: 1.0617 - val_accuracy: 0.8499
Epoch 2/15
59/59 [==============================] - 30s 515ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 1.0098 - val_accuracy: 0.8543
Epoch 3/15
59/59 [==============================] - 30s 513ms/step - loss: 0.0053 - accuracy: 0.9983 - val_loss: 1.0422 - val_accuracy: 0.8488
Epoch 4/15
59/59 [==============================] - 30s 512ms/step - loss: 0.0042 - accuracy: 0.9984 - val_loss: 1.0484 - val_accuracy: 0.8554
Epoch 5/15
59/59 [==============================] - 30s 514ms/step - loss: 0.0032 - accuracy: 0.9988 - val_loss: 1.0099 - val_accuracy: 0.8535
Epoch 6/15
59/59 [==============================] - 30s 514ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 1.0809 - val_accuracy: 0.8530
Epoch 7/15
59/59 [==============================] - 30s 516ms/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 1.1744 - val_accuracy: 0.8459
Epoch 8/15

In [30]:
results = model.evaluate(test_data.batch(256), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

98/98 - 7s - loss: 1.1667 - accuracy: 0.8409 - 7s/epoch - 71ms/step
loss: 1.167
accuracy: 0.841


ВЫВОДЫ: В первом случаем приобучении на первоначальном обучающем слое с 10 эпохами на Адам accuracy: 0.838 on 15 accuracy: 0.841
на Надам accuracy: 0.840 на 15 эпохах accuracy: 0.839
для получеия хороших результатов 90+ нужно оптимизировать количество слоев из наполненость и количество неронов в них количество эпох почти не влияет на точность моделикак и размер

In [ ]:
model = keras.Sequential([keras.layers.Flatten(input_shape=(28, 28), name='input'),
    
    keras.layers.Dense(256, activation='relu', name='hiden_one'),
    keras.layers.Dense(128, activation='relu', name='hiden_two'),
    keras.layers.Dense(64, activation='tanh', name='hiden_three'),
    keras.layers.Dense(64, activation='relu', name='hiden_four'),
    keras.layers.Dense(1, name='output')
])
#config = model.get_config()
##model.save_weights('imdb_reviews_weights.h5')
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)


In [ ]:
history = model.fit(train_data.batch(256),
                    epochs=20,
                    validation_data=validation_data.batch(256),
                    verbose=1)

После перебора всех параметров нейросети, созданной на уроке можно уверенно заключить:
- Улучшить ее работу только изменяя параметры невозможно.

- Не улучшает ситуацию последовательное обучение с использованием различных входных пакетов.

- Не удалось также создать сеть с другим количеством слоёв существенно превосходящую образец.

- Не оправдала себя идея использовать для обучения зеркально (горизонтально или вертикально) отраженные изображения.

**-Единственный вариант давший небольшие улучшения это использование комбинации 2х сетей, сети образца и сети с параметрами, которая "заточена" максимально хорошо предсказывать самый плохо определяемый класс.**